In [1]:
import firebase_admin
from firebase_admin import credentials , storage
from firebase_admin import db
import pandas as pd 
import os
import dotenv
dotenv.load_dotenv()
from supabase import create_client

## Firebase Init

In [2]:
# url = os.getenv('SUPABASE_URL')
# key = os.getenv('SUPABASE_KEY')
# supabase = create_client(url, key)

In [3]:
service_account_info ={
    "type": os.getenv('FIREBASE_TYPE'),
    "project_id": os.getenv('FIREBASE_PROJECT_ID'),
    "private_key_id": os.getenv('FIREBASE_PRIVATE_KEY_ID'),
    "private_key": os.getenv('FIREBASE_PRIVATE_KEY'),
    "client_email": os.getenv('FIREBASE_CLIENT_EMAIL'),
    "client_id": os.getenv('FIREBASE_CLIENT_ID'),
    "auth_uri": os.getenv('FIREBASE_AUTH_URI'),
    "token_uri": os.getenv('FIREBASE_TOKEN_URI'),
    "auth_provider_x509_cert_url": os.getenv('FIREBASE_AUTH_PROVIDER_X509_CERT_URL'),
    "client_x509_cert_url": os.getenv('FIREBASE_CLIENT_X509_CERT_URL'),
    "universe_domain": os.getenv('FIREBASE_UNIVERSE_DOMAIN')
  }

In [ ]:
cred = credentials.Certificate(service_account_info)
firebase_admin.initialize_app(cred,{
    'storageBucket': 'coffee-shop-app-a4152.appspot.com',
     'databaseURL': 'https://coffee-shop-d6777-default-rtdb.firebaseio.com/'
})

### Upload Data

In [32]:
image_folder_path = './products/images/'

In [33]:
df = pd.read_json('products/products.jsonl', lines=True)
df.head(2)


,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [34]:
def upload_image(bucket_name, image_path):
    image_name = image_path.split('/')[-1]
    file_path_in_bucket = f'product_images/{image_name}'
    
    # Check if the file already exists
    existing_file = supabase.storage.from_(bucket_name).list(path='product_images/', search=image_name)
    if existing_file:
        print(f"{image_name} already exists. Skipping upload.")
    else:
        with open(image_path, "rb") as f:
            response = supabase.storage.from_(bucket_name).upload(file_path_in_bucket, f)
    
    # Generate public URL
    public_url = f"{url}/storage/v1/object/public/{bucket_name}/{file_path_in_bucket}"
    return public_url

In [35]:
for index, row in df.iterrows():
    print(index, row['name'])
    
    image_path = os.path.join(image_folder_path, row['image_path'])
    image_url = upload_image('products', image_path)  # 'products' is the bucket name
    
    product_data = row.to_dict()
    product_data.pop('image_path')  # Remove image_path from the data
    product_data['image_url'] = image_url  # Add image_url for the product
    
    # Ensure product_data matches the table schema
    try:
        # Insert data into Supabase table 'products'
        response = supabase.table('products').insert(product_data).execute()
        if response.status_code == 201:
            print(f"Product {row['name']} uploaded successfully.")
        else:
            print(f"Error uploading {row['name']}: {response.status_code}")
    except Exception as e:
        print(f"An error occurred while inserting product {row['name']}: {str(e)}")

0 Cappuccino


TypeError: SyncBucketActionsMixin.list() got an unexpected keyword argument 'search'

In [47]:
from pymongo import MongoClient
import gridfs
import pandas as pd
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# MongoDB Atlas connection URI (Replace with your MongoDB Atlas URI)
MONGO_URI = os.getenv('MONGO_URI')

# Connect to MongoDB Atlas
client = MongoClient(MONGO_URI)
db = client['product_db']  # Database name
fs = gridfs.GridFS(db)  # GridFS for handling large files

# Folder paths
image_folder_path = './products/images/'

# Load product data
df = pd.read_json('products/products.jsonl', lines=True)
print(df.head(2))

# Function to upload image to GridFS
def upload_image(image_path):
    image_name = image_path.split('/')[-1]
    with open(image_path, "rb") as img_file:
        # Upload the image to GridFS and get its file_id
        file_id = fs.put(img_file, filename=image_name)
        print(f"Image '{image_name}' uploaded successfully with ID: {file_id}")
    return file_id

# Function to generate public URL (you can serve this with a custom server or use MongoDB Data API)
def generate_image_url(file_id):
    # Since MongoDB itself doesn't provide a public URL, we assume you would use a custom server for serving the image
    return f"/image/{file_id}"

# Upload images and store product data in MongoDB
for index, row in df.iterrows():
    print(index, row['name'])
    
    # Image path
    image_path = os.path.join(image_folder_path, row['image_path'])
    
    # Upload image to GridFS
    file_id = upload_image(image_path)
    
    # Prepare product data for insertion into MongoDB
    product_data = row.to_dict()
    product_data.pop('image_path')  # Remove image_path from the data
    product_data['image_file_id'] = file_id  # Store file_id from GridFS
    product_data['image_url'] = generate_image_url(file_id)  # Generate image URL
    
    # Insert product data into the 'products' collection
    try:
        response = db.products.insert_one(product_data)
        print(f"Product {row['name']} uploaded successfully with ID: {response.inserted_id}")
    except Exception as e:
        print(f"An error occurred while inserting product {row['name']}: {str(e)}")

                 name category  \
0          Cappuccino   Coffee   
1  Jumbo Savory Scone   Bakery   

                                         description  \
0  A rich and creamy cappuccino made with freshly...   
1  Deliciously flaky and buttery, this jumbo savo...   

                                         ingredients  price  rating  \
0                [Espresso, Steamed Milk, Milk Foam]   4.50     4.7   
1  [Flour, Butter, Cheese, Herbs, Baking Powder, ...   3.25     4.3   

         image_path  
0    cappuccino.jpg  
1  SavoryScone.webp  
0 Cappuccino
Image 'cappuccino.jpg' uploaded successfully with ID: 679e0c3c60d1ee3f289327fd
Product Cappuccino uploaded successfully with ID: 679e0c3d60d1ee3f289327ff
1 Jumbo Savory Scone
Image 'SavoryScone.webp' uploaded successfully with ID: 679e0c3d60d1ee3f28932800
Product Jumbo Savory Scone uploaded successfully with ID: 679e0c3e60d1ee3f28932805
2 Latte
Image 'Latte.jpg' uploaded successfully with ID: 679e0c3e60d1ee3f28932806
Product Latte 